查詢資料庫

In [6]:
import os
import pymysql
from dotenv import load_dotenv

load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")


# 進階條件（同時符合）
advanced_keywords = ["uno"]

def query_advanced_results():
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4'
    )

    with connection:
        with connection.cursor() as cursor:
            sql = "SELECT title, price, unit_price, full_text, timestamp FROM coupang_products;"
            cursor.execute(sql)
            rows = cursor.fetchall()

            print(
                f"\n進階查詢（包含：{'、'.join(advanced_keywords)}）結果：\n"
            )
            for row in rows:
                full_text = row[3]
                if all(kw in full_text for kw in advanced_keywords):
                    print(f"標題: {row[0]}")
                    print(f"價格: {row[1]}")
                    print(f"每單位: {row[2]}")
                    print(f"完整內容: {row[3]}")
                    print(f"時間戳記: {row[4]}")
                    print("-" * 60)

if __name__ == "__main__":
    query_advanced_results()



進階查詢（包含：uno）結果：



檢查重複

In [3]:
# 進階條件（同時符合）
advanced_keywords = ["味丹 多喝水", "檸檬風味"]

def query_advanced_results():
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4'
    )

    # 記錄已顯示的完整內容
    seen_full_text = set()

    with connection:
        with connection.cursor() as cursor:
            sql = "SELECT title, price, unit_price, full_text, timestamp FROM coupang_products ORDER BY timestamp DESC;"
            cursor.execute(sql)
            rows = cursor.fetchall()

            print(f"\n進階查詢（包含：{'、'.join(advanced_keywords)}）結果：\n")
            for row in rows:
                title, price, unit_price, full_text, timestamp = row

                # 進階條件比對 & 重複內容排除
                if all(kw in full_text for kw in advanced_keywords):
                    if full_text in seen_full_text:
                        # 跳過重複內容
                        continue
                    seen_full_text.add(full_text)

                    print(f"標題: {title}")
                    print(f"價格: {price}")
                    print(f"每單位: {unit_price}")
                    print(f"完整內容: {full_text}")
                    print(f"時間戳記: {timestamp}")
                    print("-" * 60)

if __name__ == "__main__":
    query_advanced_results()



進階查詢（包含：味丹 多喝水、檸檬風味）結果：

標題: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 24瓶
價格: $287
每單位: ($2.14/100ml)
完整內容: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 24瓶 特價 46折 $624 $287 ($2.14/100ml) 7折 優惠券 明天 5/13 (二) 預計送達 免運 滿 $490 ( 909 )
時間戳記: 2025-05-12 00:53:31
------------------------------------------------------------
標題: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 48瓶
價格: $674
每單位: ($2.51/100ml)
完整內容: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 48瓶 特價 55折 $1,248 $674 ($2.51/100ml) 7折 優惠券 明天 5/13 (二) 預計送達 免運 ( 909 )
時間戳記: 2025-05-12 00:53:31
------------------------------------------------------------
標題: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 72瓶
價格: $1,061
每單位: ($2.63/100ml)
完整內容: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 72瓶 特價 57折 $1,872 $1,061 ($2.63/100ml) 7折 優惠券 明天 5/13 (二) 預計送達 免運 ( 909 )
時間戳記: 2025-05-12 00:53:31
------------------------------------------------------------


輸出資料庫全部內容

In [ ]:
def query_all_products():
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4'
    )

    with connection:
        with connection.cursor() as cursor:
            sql = "SELECT id, search_keyword, title, price, unit_price, full_text, timestamp FROM coupang_products ORDER BY id DESC;"
            cursor.execute(sql)
            rows = cursor.fetchall()

            print("資料庫 coupang_products 全部內容：\n")
            for row in rows:
                print(f"ID: {row[0]}")
                print(f"關鍵字: {row[1]}")
                print(f"標題: {row[2]}")
                print(f"價格: {row[3]}")
                print(f"每單位: {row[4]}")
                print(f"完整內容: {row[5]}")
                print(f"時間戳記: {row[6]}")
                print("-" * 80)

if __name__ == "__main__":
    query_all_products()


資料查詢並匯出

In [ ]:
import pandas as pd
from datetime import datetime

def query_products(
    start_date=None, 
    end_date=None, 
    limit=None, 
    offset=None,
    # 如：'output.csv' 或 'output.xlsx'
    export_path=None
):
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4',
        # 回傳 dict 格式
        cursorclass=pymysql.cursors.DictCursor
    )

    with connection:
        with connection.cursor() as cursor:
            sql = "SELECT id, search_keyword, title, price, unit_price, full_text, timestamp FROM coupang_products WHERE 1=1"
            params = []

            # 日期篩選
            if start_date and end_date:
                sql += " AND timestamp BETWEEN %s AND %s"
                params.extend([start_date, end_date])

            # 排序
            sql += " ORDER BY id DESC"

            # 分頁功能
            if limit:
                sql += " LIMIT %s"
                params.append(limit)
                if offset:
                    sql += " OFFSET %s"
                    params.append(offset)

            cursor.execute(sql, params)
            rows = cursor.fetchall()

            # 顯示資料
            print("查詢結果：")
            for row in rows:
                print(f"ID: {row['id']}")
                print(f"關鍵字: {row['search_keyword']}")
                print(f"標題: {row['title']}")
                print(f"價格: {row['price']}")
                print(f"每單位: {row['unit_price']}")
                print(f"完整內容: {row['full_text']}")
                print(f"時間戳記: {row['timestamp']}")
                print("-" * 80)

            # 匯出功能
            if export_path:
                df = pd.DataFrame(rows)
                if export_path.endswith(".csv"):
                    df.to_csv(
                        export_path,
                        index=False,
                        encoding="utf-8-sig"
                    )
                    print(f"資料已匯出為 CSV：{export_path}")
                elif export_path.endswith(".xlsx"):
                    df.to_excel(export_path, index=False)
                    print(f"資料已匯出為 Excel：{export_path}")
                else:
                    print("不支援的檔案格式，只支援 .csv 與 .xlsx")

if __name__ == "__main__":
    # 篩選日期（可調整）
    start_date = "2024-12-01"
    end_date = "2025-12-31"

    # 分頁參數（可調整）
    limit = 10
    offset = 0

    # 匯出路徑（None 表示不匯出）
    # 或 coupang_output.csv
    export_path = "coupang_output.xlsx"

    # 執行查詢
    query_products(
        start_date=start_date,
        end_date=end_date,
        limit=limit,
        offset=offset,
        export_path=export_path
    )
